

```
# This is formatted as code
```

**Llama 2+ Chroma + LangChain**
Step 1: Install All the Required Pakages

In [ ]:
!pip install langchain
!pip install pypdf
!pip install unstructured
!pip install sentence_transformers
!pip install chromadb
!pip install llama-cpp-python
!pip install huggingface_hub
!pip install -qq langchain wget
!pip install gguf  #https://github.com/ggerganov/llama.cpp/tree/master/gguf-py
!git clone https://github.com/ggerganov/llama.cpp
!pip -qq install git+https://github.com/huggingface/transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.6/276.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.1/275.1 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 27.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 39.6 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=e0d484d1d450d2b516d4aed1676ee39a6757e9465f3bf7ed3d40e55581822563
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a

***Step 2: Import All the Required Libraries***

In [ ]:
from langchain.document_loaders import PyPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from sentence_transformers import SentenceTransformer
from langchain.chains.question_answering import load_qa_chain
import os

***Step 3: Load the Data***

In [ ]:
loader = PyPDFLoader("/content/bill.pdf")

In [ ]:
data = loader.load()

In [ ]:
len(data)

221

***Step 4: Split the Text into Chunks***

In [ ]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)

In [ ]:
docs=text_splitter.split_documents(data)

In [ ]:
len(docs)

1829

In [ ]:
docs[0]

Document(page_content='BILL No. 18 OF 2016\nTHE  FINANCE BILL, 2016\n(AS  INTRODUCED  IN  LOK  SABHA)', metadata={'source': '/content/bill.pdf', 'page': 0})

***Step 5: Downlaod the Embeddings***

In [ ]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

***Step 6 : creating the chroma***

In [ ]:
db = Chroma.from_documents(docs, embeddings, persist_directory="db")

***Step 7: Create Embeddings for Each of the Text Chunk***

In [ ]:
docsearch=Chroma.from_texts([t.page_content for t in docs], embeddings, persist_directory="db")

***Step 8: Similarity Search***

In [ ]:
query="What is the Selection Status of Candidate Name Abhijit M?"

In [ ]:
docs=docsearch.similarity_search(query, k=1)

In [ ]:
docs

[Document(page_content='Please find below feedback :  \nCandidate Name : Abhijit M  \nInterview Type : L1  \nSelection Status : Yes')]

**Step 9 = Query the Docs to get the Answer Back (Llama 2 Model)**

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip -qq install --upgrade --force-reinstall llama-cpp-python --no-cache-dir


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 13.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 107.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 168.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cupy-cuda11x 11.0.0 requires numpy<1.26,>=1.20, but you have numpy 1.26.0 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.26.0 which is incompatible.
tensorflow 2.13.0 requires numpy<=1.24.3,>=1.22, but you have numpy 1.26.0 which is incompatible.
tensorflow 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.8.0 which is incompatible.


**Import All the Required Libraries**


In [ ]:
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from huggingface_hub import hf_hub_download
from langchain.chains.question_answering import load_qa_chain


In [ ]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

**Quantized Models from the Hugging Face Community**

In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

In [ ]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

In [ ]:
n_gpu_layers = 40  # Change this value based on your model and your GPU VRAM pool.
n_batch = 256  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.

# Loading model,
llm = LlamaCpp(
model_path=model_path,
max_tokens=256,
n_gpu_layers=n_gpu_layers,
n_batch=n_batch,
callback_manager=callback_manager,
n_ctx=1024,
verbose=True,
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [ ]:
chain=load_qa_chain(llm, chain_type="stuff")


In [ ]:
query="What is the Selection Status of Candidate Name Abhijit M ?"
docs=docsearch.similarity_search(query, k=1)

In [ ]:
docs

[Document(page_content='Please find below feedback :  \nCandidate Name : Abhijit M  \nInterview Type : L1  \nSelection Status : Yes')]

In [ ]:
chain.run(input_documents=docs, question=query)

 The Selection Status of Candidate Name Abhijit M is "Yes".

' The Selection Status of Candidate Name Abhijit M is "Yes".'